In [1]:
import cv2
import csv
import mediapipe as mp
import numpy as np
from collections import deque
import os

In [2]:
mp_draw = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
num_coords = 21 * 2

In [4]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [5]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [ ]:
# Inisialisasi variabel
cvFpsCalc = cv2.TickMeter()

history_length = 16
point_history = deque(maxlen=history_length)
finger_gesture_history = deque(maxlen=history_length)

cap = cv2.VideoCapture(0)


file_path = 'datahurufbisindo.csv'

class_label = input("Masukkan kelas: ")

recorded_data = 0

# Definisi landmarks jika belum ada
if not os.path.isfile(file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(landmarks)

# Membuat variabel untuk menandai sudah mencatat data atau belum
data_recorded = False

with open(file_path, mode='a', newline='') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image_rgb.flags.writeable = False

            results = holistic.process(image_rgb)

            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            image_bgr.flags.writeable = True

            if results.right_hand_landmarks:
                mp_draw.draw_landmarks(image_bgr, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                       mp_draw.DrawingSpec(color=(65, 145, 151), thickness=2, circle_radius=4),
                                       mp_draw.DrawingSpec(color=(245, 252, 205), thickness=2, circle_radius=2))

            if results.left_hand_landmarks:
                mp_draw.draw_landmarks(image_bgr, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                       mp_draw.DrawingSpec(color=(65, 145, 151), thickness=2, circle_radius=4),
                                       mp_draw.DrawingSpec(color=(245, 252, 205), thickness=2, circle_radius=2))

            try:
                right_hand = results.right_hand_landmarks.landmark
                left_hand = results.left_hand_landmarks.landmark
                
                right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for
                                                landmark in right_hand]).flatten())
                left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for
                                               landmark in left_hand]).flatten())

                row = right_hand_row + left_hand_row
                row.insert(0, class_label)
                
                writer.writerow(row)
                recorded_data += 1 

                if recorded_data >= 100:
                    print("Telah mencapai 100 data. Perekaman dihentikan.")
                    cap.release()  # Memberhentikan perekaman dengan melepaskan kamera
                    cv2.destroyAllWindows()  # Menutup jendela
                    break
            
            except:
                pass

            cv2.imshow('Kamera', image_bgr)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [35]:
df = pd.read_csv('datahurufbisindo.csv')

In [36]:
dept_emp_num = df.groupby('class')['class'].count()
total_kelas = df.groupby('class')['class'].count().sum()
print (dept_emp_num)
print("Total Keseluruhan Kelas:", total_kelas)
df.shape

class
A    100
B    100
C    100
D    100
E    100
F    100
G    100
H    100
I    100
J    100
K    100
L    100
M    100
N    100
O    100
P    100
Q    100
R    100
S    100
T    100
U    100
V    100
W    100
X    100
Y    100
Name: class, dtype: int64
Total Keseluruhan Kelas: 2500


(2500, 169)

In [37]:
# Menghapus baris tertentu dari file CSV
startline, endline = 2502, 52053
filename1= 'datahurufbisindo.csv'

# Membaca file CSV dan menyimpan kontennya ke dalam list content
with open(filename1, 'r', newline='') as f:
    content = [row for i,row in enumerate(csv.reader(f), 1)
                    if i not in range(startline, endline+1)]
filename2 = 'datahurufbisindo.csv'  

# Menulis kembali ke file CSV setelah menghapus baris
with open(filename2, 'w', newline='') as f:
    csv.writer(f).writerows(content)

print('lines deleted')

lines deleted


In [38]:
df.head()  # Menampilkan lima baris pertama dari DataFrame

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z40,v40,x41,y41,z41,v41,x42,y42,z42,v42
0,A,0.180874,0.848098,1.765462e-07,0.0,0.256062,0.826749,-0.009746,0.0,0.315121,...,-0.072543,0.0,0.641025,0.732655,-0.060201,0.0,0.644885,0.769729,-0.045585,0.0
1,A,0.209353,0.770274,7.335093e-09,0.0,0.275673,0.744213,-0.007578,0.0,0.331933,...,-0.063948,0.0,0.633523,0.625981,-0.049775,0.0,0.643830,0.657332,-0.034968,0.0
2,A,0.213697,0.709036,2.986468e-08,0.0,0.276213,0.690909,-0.011912,0.0,0.329810,...,-0.055940,0.0,0.620513,0.571294,-0.045261,0.0,0.632496,0.597869,-0.031529,0.0
3,A,0.228049,0.667049,1.459347e-07,0.0,0.290988,0.649778,-0.010581,0.0,0.341338,...,-0.059955,0.0,0.615882,0.544104,-0.049525,0.0,0.632572,0.557280,-0.035898,0.0
4,A,0.238403,0.631731,1.062676e-07,0.0,0.295630,0.611347,-0.016664,0.0,0.348790,...,-0.057459,0.0,0.613982,0.511713,-0.046885,0.0,0.630567,0.523789,-0.033348,0.0


In [39]:
df.tail() # Menampilkan lima baris terakhir dari DataFrame

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z40,v40,x41,y41,z41,v41,x42,y42,z42,v42
2495,Y,0.243204,0.860766,-1.144866e-08,0.0,0.316776,0.812897,0.008230,0.0,0.359243,...,-0.016258,0.0,0.346507,0.651913,-0.013336,0.0,0.363252,0.655885,-0.007247,0.0
2496,Y,0.242630,0.860541,-1.684572e-08,0.0,0.318100,0.814446,0.008708,0.0,0.359951,...,-0.023188,0.0,0.344595,0.647364,-0.020705,0.0,0.362674,0.652075,-0.015072,0.0
2497,Y,0.242833,0.859935,-1.870569e-08,0.0,0.318645,0.813547,0.007052,0.0,0.361193,...,-0.018781,0.0,0.350652,0.650392,-0.013784,0.0,0.369854,0.656466,-0.006617,0.0
2498,Y,0.244399,0.860546,-1.741019e-08,0.0,0.320174,0.810926,0.008628,0.0,0.362638,...,-0.005054,0.0,0.345580,0.665029,-0.000946,0.0,0.358987,0.669066,0.004584,0.0
2499,Y,0.245479,0.859932,-8.251513e-09,0.0,0.320050,0.811133,0.007827,0.0,0.362526,...,-0.021916,0.0,0.345952,0.658427,-0.017961,0.0,0.363534,0.663667,-0.011556,0.0


In [40]:
X = df.drop('class', axis=1) # features akan menjadi semua nilai koord
y = df['class'] # target = label kelas

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1234) #pembagian dataset

In [42]:
X_test

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z40,v40,x41,y41,z41,v41,x42,y42,z42,v42
903,0.096925,0.646085,-1.872206e-07,0.0,0.179784,0.620224,-0.001460,0.0,0.243075,0.584887,...,-0.031514,0.0,0.827854,0.399934,-0.032435,0.0,0.810613,0.367053,-0.028684,0.0
674,0.354526,0.487672,-2.080406e-07,0.0,0.392459,0.413232,0.004934,0.0,0.445189,0.380808,...,-0.031559,0.0,0.411789,0.731966,-0.019636,0.0,0.431334,0.728973,-0.010851,0.0
1624,0.218169,0.736256,1.206284e-07,0.0,0.264046,0.728905,-0.019693,0.0,0.314133,0.687060,...,-0.044076,0.0,0.414320,0.156890,-0.047375,0.0,0.391623,0.135332,-0.046264,0.0
1214,0.533806,0.801348,2.734025e-07,0.0,0.536322,0.705675,0.007067,0.0,0.577195,0.639211,...,-0.047307,0.0,0.865017,0.364829,-0.056866,0.0,0.881798,0.329317,-0.063258,0.0
2164,0.157087,0.705267,4.685939e-07,0.0,0.205866,0.700237,-0.029616,0.0,0.242436,0.650985,...,-0.072707,0.0,0.729079,0.580725,-0.067387,0.0,0.728293,0.616231,-0.057224,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,0.142557,0.546195,4.911705e-07,0.0,0.182850,0.461411,0.012131,0.0,0.246145,0.433231,...,-0.058320,0.0,0.667802,0.538310,-0.058734,0.0,0.665240,0.530068,-0.058593,0.0
1084,0.312525,0.318549,-1.046604e-07,0.0,0.367830,0.391532,0.003757,0.0,0.419572,0.426478,...,-0.066718,0.0,0.544406,0.579044,-0.059730,0.0,0.563159,0.586832,-0.048061,0.0
849,0.216967,0.627890,-3.200064e-07,0.0,0.265639,0.630990,-0.017570,0.0,0.318547,0.593870,...,-0.025895,0.0,0.672253,0.399979,-0.030808,0.0,0.667975,0.361791,-0.031172,0.0
954,0.139712,0.453364,-1.383469e-07,0.0,0.215904,0.438038,0.005749,0.0,0.292189,0.443429,...,-0.072927,0.0,0.659268,0.648567,-0.077958,0.0,0.637046,0.688281,-0.081526,0.0


In [43]:
y_test

903     J
674     G
1624    Q
1214    M
2164    V
       ..
485     E
1084    K
849     I
954     J
435     E
Name: class, Length: 1250, dtype: object

In [44]:
print("Jumlah data total:", len(X))
print("Jumlah data training set:", len(X_train))
print("Jumlah data testing set:", len(X_test))

Jumlah data total: 2500
Jumlah data training set: 1250
Jumlah data testing set: 1250


In [45]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestClassifier

In [46]:
#Penggabungan langkah pemrosesan data
pipelines = {
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
}

In [47]:
#Penggunaan data pelatihan
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [48]:
#Menampilkan hasil training
fit_models 

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())])}

In [49]:
rf_predictions = fit_models['rf'].predict(X_test)

In [50]:
fit_models['rf'].predict(X_test)

array(['J', 'G', 'Q', ..., 'I', 'J', 'E'], dtype=object)

In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix # Accuracy metrics 
import pickle 

In [52]:
for algo, model in fit_models.items():
    y_prediksi = model.predict(X_test) #sebagai y prediksi
    print(algo, classification_report(y_test, y_prediksi))

    # Menghitung precision dan recall secara total
    total_precision = precision_score(y_test, y_prediksi, average='weighted')
    total_recall = recall_score(y_test, y_prediksi, average='weighted')
    total_accuracy = accuracy_score(y_test, y_prediksi)
    
    print(f"Metrik Klasifikasi untuk {algo}:")
    print(f"Total Accuracy: {total_accuracy:.2f}")
    print(f"Total Recall: {total_recall:.2f}")
    print(f"Total Precision: {total_precision:.2f}")

rf               precision    recall  f1-score   support

           A       1.00      0.98      0.99        51
           B       1.00      1.00      1.00        47
           C       1.00      1.00      1.00        55
           D       1.00      1.00      1.00        58
           E       1.00      1.00      1.00        52
           F       1.00      1.00      1.00        54
           G       1.00      1.00      1.00        47
           H       0.98      1.00      0.99        50
           I       1.00      0.94      0.97        53
           J       0.90      1.00      0.95        43
           K       1.00      0.98      0.99        45
           L       0.96      1.00      0.98        53
           M       0.98      1.00      0.99        44
           N       0.98      0.98      0.98        46
           O       1.00      0.98      0.99        57
           P       1.00      1.00      1.00        50
           Q       1.00      1.00      1.00        48
           R       1.00 

In [53]:
rf_predictions = fit_models['rf'].predict(X_test)

In [54]:
fit_models['rf'].predict(X_test)

array(['J', 'G', 'Q', ..., 'I', 'J', 'E'], dtype=object)

In [55]:
y_train

376     D
348     D
1863    S
1871    S
816     I
       ..
279     C
2041    U
664     G
1318    N
723     H
Name: class, Length: 1250, dtype: object

In [56]:
X_train

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z40,v40,x41,y41,z41,v41,x42,y42,z42,v42
376,0.237615,0.531955,1.080346e-07,0.0,0.290052,0.538109,-0.007115,0.0,0.345951,0.516657,...,-0.065652,0.0,0.441513,0.633545,-0.058556,0.0,0.460043,0.648599,-0.045682,0.0
348,0.244423,0.528913,1.114112e-07,0.0,0.296175,0.535089,-0.006799,0.0,0.352198,0.514193,...,-0.064493,0.0,0.442451,0.632366,-0.056852,0.0,0.460344,0.646064,-0.043440,0.0
1863,0.241978,0.540439,1.200154e-07,0.0,0.296097,0.531117,-0.007188,0.0,0.350079,0.493816,...,-0.068050,0.0,0.554848,0.655533,-0.057300,0.0,0.574862,0.648778,-0.046924,0.0
1871,0.242657,0.552702,1.412051e-07,0.0,0.297387,0.544728,-0.008141,0.0,0.354196,0.505393,...,-0.064787,0.0,0.540574,0.657572,-0.055085,0.0,0.562115,0.651248,-0.044778,0.0
816,0.225152,0.630451,-2.714760e-07,0.0,0.275629,0.630268,-0.012298,0.0,0.327424,0.586618,...,-0.018815,0.0,0.650830,0.403075,-0.022663,0.0,0.644940,0.368781,-0.022288,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,0.175853,0.505529,1.370493e-07,0.0,0.233821,0.516186,-0.012646,0.0,0.301430,0.500421,...,-0.057433,0.0,0.713669,0.423903,-0.041731,0.0,0.732882,0.423010,-0.027074,0.0
2041,0.279266,0.712409,1.174678e-07,0.0,0.312659,0.638044,-0.006191,0.0,0.321179,0.553504,...,-0.051704,0.0,0.623831,0.611240,-0.036703,0.0,0.625090,0.639286,-0.021722,0.0
664,0.354539,0.487757,-2.414464e-07,0.0,0.391304,0.413525,0.005346,0.0,0.445105,0.378941,...,-0.039821,0.0,0.415733,0.726091,-0.029679,0.0,0.437234,0.721598,-0.021380,0.0
1318,0.730136,0.622888,3.615573e-07,0.0,0.674995,0.595143,-0.015497,0.0,0.630089,0.538365,...,-0.038178,0.0,0.800884,0.348534,-0.044243,0.0,0.813587,0.316330,-0.048587,0.0


In [57]:
print(X_train.shape)
print(X_test.shape)

(1250, 168)
(1250, 168)


In [58]:
with open('modelhurufbisindo.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [59]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [60]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import pandas as pd

In [61]:
# Inisialisasi model Random Forest dari file PKL
with open('modelhurufbisindo.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False

        results = holistic.process(image_rgb)

        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        image_bgr.flags.writeable = True

        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(image_bgr, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(65, 145, 151), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(245, 252, 205), thickness=2, circle_radius=2))

        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(image_bgr, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(65, 145, 151), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(245, 252, 205), thickness=2, circle_radius=2))

        try:
            right_hand = results.right_hand_landmarks.landmark
            left_hand = results.left_hand_landmarks.landmark

            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for
                                           landmark in right_hand]).flatten())
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for
                                          landmark in left_hand]).flatten())

            row = right_hand_row + left_hand_row

            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]  # Prediksi label
            body_language_prob = model.predict_proba(X)[0]
            
            coords = (30, 50)  # Koordinat untuk menampilkan tag

            # Membuat kotak untuk tulisan prediksi
            cv2.rectangle(image_bgr, (coords[0], coords[1] + 5), 
                          (coords[0] + len(f'Prediksi: {body_language_class}') * 20, coords[1] - 30),
                          (0, 255, 0), -1)
            
            # Menambahkan teks prediksi ke frame
            cv2.putText(image_bgr, f'Prediksi: {body_language_class}', coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        except:
            pass

        cv2.imshow('Kamera', image_bgr)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


/home/userxaf/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/userxaf/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/userxaf/.local/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/userxaf/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is